# Producer Development


### Notebook for testing out the interface to Twitter and IEX

In [21]:
#Import
import numpy as np
import pandas as pd

#Twitter requirements
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

#IEX requirements
import urllib.request


import json

### Create Company List

In [22]:
companies = {
    "Apple":"AAPL",
    "Facebook":"FB",
    "Google":"GOOG"
    }

#company_df = pd.DataFrame(companies.items(), columns=['Company Name','Ticker'])
company_df = pd.DataFrame.from_dict(companies, orient='index')
company_df.index.name = 'Company'
company_df.columns=['Ticker']
company_df.head()

,Ticker
Company,
Apple,AAPL
Facebook,FB
Google,GOOG


In [30]:
#Write to CSV
company_df.to_csv('companies.csv')

### Twitter Ingestion

In [23]:
#Created a twitter tokens file. That has the tokens. This will not be in GIT
from twitter_tokens import *

In [24]:
#Add Code to read CSV file. In order to account for changes in the list, have to restart
companies = pd.read_csv('companies.csv')

#Add code to add ticker symbol
companies['tweet_ticker']=companies['Ticker'].apply(lambda x: '$'+x)
companies

,Company,Ticker,tweet_ticker
0,Apple,AAPL,$AAPL
1,Facebook,FB,$FB
2,Google,GOOG,$GOOG


In [44]:
tickers = companies['tweet_ticker'].tolist()
#Tweets with the ticker in front come in very slowly. May take a while to build up. But these are official tweets

In [46]:
#Need to add in reading csv file
#Test

#This is a basic listener that just prints received tweets to stdout.
class TweetListener(StreamListener):
    global count
    count = 0
    def on_data(self, data):
        try:
            datajson = json.loads(data)
            global count
            count += 1
            print(count)
            print(datajson['text'])
            #coll_reference.insert_one(datajson)
            return True
        except Exception as e:
            print(e)

    def on_error(self, status):
        print('The error code is: ' + repr(status))
        #Continue even if there is an error
        return True

#This handles Twitter authetication and the connection to Twitter Streaming API
tweetlist = TweetListener(api=API(wait_on_rate_limit=True))
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
stream = Stream(auth, tweetlist)

#Filters by the ticker names
print('Filtering: ' + str(tickers))
stream.filter(track=tickers)

Filtering: ['$AAPL', '$FB', '$GOOG']
1
RT @OphirGottlieb: $AAPL When Apple changes something that we don't realize we want, it takes time for that realization to set in.

It took…
2
Legally benefiting from insider trading! My strategy revealed! https://t.co/UUoeswqecB $NVEE $MZOR $FB $TWTR $STUDY $EYES
3
RT @Sandro_power: $PYPL  People it will be one thing if someone esle likes a stock, but when PowerTriggerTrade speaks about it with a plan…
4
$AAPL Former Apple Exec Fadell on Impact of Tech Addiction via https://t.co/jURZDymPXI
5
RT @DrivingTheDay: Is Brand Loyalty Dead? (infographic) https://t.co/hihoNLPH6V #Google $GOOG $GOOGL #Amazon $AMZN #Apple $AAPL #stocks #in…
6
RT @Sandro_power: People the steam from the nose of the bull acts like a rocket launcher for the money arrow going higher. Something to thi…
7
Wonder if @OphirGottlieb has jumped into $aapl here 

Currently probing the lower edge of its recent range -- $amzn… https://t.co/zT8k19WrhB
8
$FB : bcb29b96-557b-4322-b4d0-5395

KeyboardInterrupt: 